# YOLOv8 Finetuning - Getting Started

This notebook demonstrates how to:
1. Verify GPU setup
2. Explore the dataset
3. Train a model
4. Evaluate results

## 1. Setup and Imports

In [ ]:
import torch
import os
from pathlib import Path
from ultralytics import YOLO
import yaml
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2. Dataset Exploration

In [ ]:
# Load dataset config
with open('../config/dataset.yaml', 'r') as f:
    dataset_config = yaml.safe_load(f)

print("Dataset Configuration:")
print(f"  Classes: {dataset_config['nc']}")
print(f"  Names: {dataset_config['names']}")
print(f"  Train path: {dataset_config['train']}")
print(f"  Val path: {dataset_config['val']}")

In [ ]:
# Count dataset images
def count_dataset():
    for split in ['train', 'val', 'test']:
        img_dir = Path(f'../datasets/images/{split}')
        lbl_dir = Path(f'../datasets/labels/{split}')
        
        if img_dir.exists():
            images = list(img_dir.glob('*.jpg')) + list(img_dir.glob('*.png'))
            labels = list(lbl_dir.glob('*.txt')) if lbl_dir.exists() else []
            print(f"{split:6s}: {len(images):4d} images, {len(labels):4d} labels")

count_dataset()

In [ ]:
# Visualize sample images with annotations
def visualize_sample(split='train', num_samples=4):
    img_dir = Path(f'../datasets/images/{split}')
    lbl_dir = Path(f'../datasets/labels/{split}')
    
    images = list(img_dir.glob('*.jpg')) + list(img_dir.glob('*.png'))
    
    if len(images) == 0:
        print(f"No images found in {img_dir}")
        return
    
    samples = np.random.choice(images, min(num_samples, len(images)), replace=False)
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    axes = axes.flatten()
    
    for idx, img_path in enumerate(samples):
        img = Image.open(img_path)
        axes[idx].imshow(img)
        axes[idx].set_title(img_path.name)
        axes[idx].axis('off')
        
        # Load and display label info
        lbl_path = lbl_dir / f"{img_path.stem}.txt"
        if lbl_path.exists():
            with open(lbl_path) as f:
                n_objects = len(f.readlines())
            axes[idx].text(10, 30, f"{n_objects} objects", 
                          bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()

# Uncomment to visualize (requires dataset)
# visualize_sample('train')

## 3. Model Training

In [ ]:
# Load model
model_path = '../model.pt'
if os.path.exists(model_path):
    model = YOLO(model_path)
    print(f"Loaded existing model: {model_path}")
else:
    model = YOLO('yolov8n.pt')
    print("Loaded pretrained YOLOv8n model")

# Model info
print(f"Model parameters: {sum(p.numel() for p in model.model.parameters()):,}")

In [ ]:
# Training (small example)
# Uncomment to run training
"""
results = model.train(
    data='../config/dataset.yaml',
    epochs=10,  # Small test run
    batch=16,
    imgsz=640,
    device=0,
    project='../runs/train',
    name='notebook_exp',
    verbose=True,
)
"""
pass

## 4. Validation and Testing

In [ ]:
# Validate model
# Uncomment to run validation
"""
metrics = model.val(
    data='../config/dataset.yaml',
    split='val',
    imgsz=640,
)

print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
"""
pass

In [ ]:
# Test on single image
# Uncomment to run inference
"""
test_image = '../datasets/images/val/example.jpg'  # Update with actual image
results = model.predict(test_image, conf=0.5)

# Visualize
for r in results:
    img = r.plot()
    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.show()
"""
pass

## 5. Export Model

In [ ]:
# Export to different formats
# Uncomment to export
"""
# ONNX for cross-platform inference
model.export(format='onnx')

# TensorRT for optimized GPU inference
model.export(format='engine')

# TorchScript
model.export(format='torchscript')
"""
pass

## Next Steps

1. Prepare your dataset in `datasets/images/` and `datasets/labels/`
2. Run a small training test (10 epochs)
3. Check TensorBoard for metrics: http://localhost:6006
4. Adjust hyperparameters in `config/training_config.yaml`
5. Run full training with `train.py`
6. Export best model for production use